# Information Retrieval and Web Analytics

# Part 1: Text Processing


In [1]:
# mount google drive if using google collab, else skip
# we are not using it because it is more comfortable to use jupyter lab

try:
    from google.colab import drive
    drive.mount('/content/drive')
    
except ModuleNotFoundError:
    pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# required imports for the notebook

import json
import csv

import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#TODO documentacion
# get tweets
with open("drive/MyDrive/data/tw_hurricane_data.json", "r") as tweet_file:
    raw_tweets = [json.loads(tweet) for tweet in tweet_file]
    
# get dictionary to map tweet ids to doc ids
# we know the ids file is a list of [doc_id \t tweet_id]
with open("drive/MyDrive/data/tweet_document_ids_map.csv", "r") as id_file:
    ids = csv.reader(id_file, delimiter="\t")
    dict_ids = {id_to_id[1]: id_to_id[0] for id_to_id in list(ids)}
    
# map tweet ids with doc ids
for tweet in raw_tweets:
    tweet_id = str(tweet["id"])
    tweet["id"] = dict_ids[tweet_id]
    
print(raw_tweets[0]["id"])

doc_1


In [4]:
# list(raw_tweets[0]["entities"]["hashtags"][0]["text"])

In [5]:
import pandas as pd
df = pd.read_json('drive/MyDrive/data/tw_hurricane_data.json',lines=True)

In [6]:
new = df[['id','full_text', 'user','created_at','entities', 'favorite_count', 'retweet_count']].copy()

In [8]:
url=[]
has=[]
user=[]
for ele in range(len(new)):
  has.append(new['entities'][ele]['hashtags'][0]["text"])
  user.append(new['user'][ele]['name'])
  try:
    url.append(new['entities'][ele]['media'][0]["expanded_url"])
  except: 
    url.append('')
new['url'] = url
new['hashtags'] = has
new['user'] = user

In [10]:
new.drop(['entities'], axis=1)

,id,full_text,user,created_at,favorite_count,retweet_count,hashtags,url
0,1575918182698979328,So this will keep spinning over us until 7 pm…...,Suz👻,2022-09-30 18:39:08+00:00,0,0,HurricaneIan,https://twitter.com/suzjdean/status/1575918182...
1,1575918151862304768,Our hearts go out to all those affected by #Hu...,Lytx,2022-09-30 18:39:01+00:00,0,0,HurricaneIan,
2,1575918140839673873,Kissimmee neighborhood off of Michigan Ave. \n...,Christopher Heath,2022-09-30 18:38:58+00:00,0,0,HurricaneIan,https://twitter.com/CHeathWFTV/status/15759181...
3,1575918135009738752,I have this one tree in my backyard that scare...,alex ✨,2022-09-30 18:38:57+00:00,0,0,scwx,
4,1575918119251419136,@AshleyRuizWx @Stephan89441722 @lilmizzheidi @...,Tess 💋,2022-09-30 18:38:53+00:00,0,0,HurricaneIan,
...,...,...,...,...,...,...,...,...
3995,1575856268022992896,"The CFRD, @CarrboroPD , Carrboro Public Works,...",Carrboro Fire-Rescue,2022-09-30 14:33:06+00:00,2,0,CarrboroSafe,
3996,1575856245650919424,Why isn’t @OsceolaCountyFl listed on the @fema...,BaconBitsNews,2022-09-30 14:33:01+00:00,0,0,Kissimmee,
3997,1575856228886089728,So it really wasn't #HurricaneIan that flooded...,@jganyfl,2022-09-30 14:32:57+00:00,16,8,HurricaneIan,https://twitter.com/jganyfl1/status/1575856228...
3998,1575856226139017216,Damage in my area in Punta Gorda...a thread. I...,CJ Haddad,2022-09-30 14:32:56+00:00,2,1,HurricaneIan,https://twitter.com/haddad_cj/status/157585622...


In [11]:
def build_terms(line):
    """
    Preprocess the article text (title + body) removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    line -- string (text) to be preprocessed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    line = line.lower()
    line = line.split()  # Tokenize the text to get a list of terms
    line = [x for x in line if x not in stop_words]  # eliminate the stopwords
    line = [stemmer.stem(word) for word in line] # perform stemming (HINT: use List Comprehension)
    return line

In [12]:
# read json -> check
    '''
    Removing stop words 
    Tokenization
    Removing punctuation marks 
    Stemming
    and... anything else you think it's needed (bonus point)
    '''
# tweet id to document id ->check
# ...
# 
# ...
# tweets as Tweet | Username | Date | Hashtags | Likes | Retweets | Url

IndentationError: ignored